## TUTORIAL 23 - Beam bridge problem
**_Keywords: successive constraint method, geometrical parametrization_**

### 1. Introduction
This Tutorial addresses geometrical parametrization applied to the physical phenomena of linear deformation of a bridge. In particular, we study the elastic problem on the two-dimensional parametrized geometry $\Omega_o(\boldsymbol(\mu))$ as in the following picture:

<img src="data/beam_bridge.png" width="70%"/>

We apply the following conditions on the boundaries: homogeneous Dirichlet conditions on $\Gamma_{o,1} \cup \Gamma_{o,2}$ such that $u_2(\boldsymbol{\mu})=0$, homogeneous Dirichlet conditions on $\Gamma_{o,8}\cup\Gamma_{o,9}$ such that $u_1(\boldsymbol{\mu})=u_2(\boldsymbol{\mu})=0$, non-homogenous Neumann conditions $\boldsymbol{\sigma}\boldsymbol{n}=-\boldsymbol{n}$ on $\Gamma_{o,7}$ and homogeneous Neumann conditions on any remaining boundaries.
Here $\boldsymbol{\sigma}$ represents the stress tensor on the domain $\Omega_o(\boldsymbol{\mu})$ that is defined as follows
$$
\boldsymbol{\sigma}(\boldsymbol{u}_o;\boldsymbol{\mu})=\lambda_1(\boldsymbol{\mu})\;\text{tr}[\nabla^S\boldsymbol{u}_o]\;\boldsymbol{I}+2\;\lambda_2(\boldsymbol{\mu})\;\nabla^S\boldsymbol{u}_o,
$$
where $\nabla^S$ denotes the symmetric part of the gradient. In all regions, we assume an isotropic linear elastic material, characterized by the following Lamè constants for plane strain
$$
\lambda_1=\frac{E\nu}{(1+\nu)(1-2\nu)}, \quad 
\lambda_2=\frac{E}{2(1-\nu)},
$$ with $E=1$ and $\nu=0.3$.

This problem is characterized by two parameters: the first parameter $\mu_0$ controls the distance between the left support $\mathcal{P}^1_o$ and the left side of the bridge, and the distance between the right support $\mathcal{P}^2_o$ and the right side of the bridge. The second parameter $\mu_1$ characterizes the Young's modulus in all regions. 

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0, \mu_1)
$$
on the parameter domain
$$
[0.05,0.6] \times [0.1,10].
$$

In order to obtain a faster (yet, provably accurate) approximation of the problem, and avoiding _any_ remeshing, we pursue a model reduction by means of a certified reduced basis reduced order method from a fixed reference domain.

### 2. Parametrized formulation

Let $\boldsymbol{u}_o(\boldsymbol{\mu})$ be the displacement of the idealized contact problem with friction under the applied load, satisfying the plane-strain linear elasticity equation in $\Omega_o$.

We will directly provide a weak formulation for this problem: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $\boldsymbol{u}_o(\boldsymbol{\mu})\in\mathbb{V}_o$ such that

$$a\left(\boldsymbol{u}_o(\boldsymbol{\mu}),\boldsymbol{v}_o;\boldsymbol{\mu}\right)=f_o(\boldsymbol{v}_o;\boldsymbol{\mu})\quad \forall \boldsymbol{v}_o\in\mathbb{V}_o$$

where

* the function space $\mathbb{V}_o$ is defined as
$$
\mathbb{V}_o=\{v_o \in H^1(\Omega): v\mid_{\Gamma_{o,1}\cup\Gamma_{o,2}\cup\Gamma_{o,8}\cup\Gamma_{o,9}} =0\}
$$

* the parametrized bilinear form $a_o(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V}_o \times \mathbb{V}_o \to \mathbb{R}$ is defined by
$$a(\boldsymbol{u}_o,\boldsymbol{v}_o;\boldsymbol{\mu})=\int_{\Omega_o} {\lambda_1(\boldsymbol{\mu})\;\text{tr}[\nabla^S\boldsymbol{u}_o]\;\text{tr}[\nabla^S\boldsymbol{v_o}]+2\;\lambda_2(\boldsymbol{\mu})\;\nabla^S\boldsymbol{u}_o:\nabla^S\boldsymbol{v}_o} \ d\boldsymbol{x},$$
* the parametrized linear form $f_o(\cdot; \boldsymbol{\mu}): \mathbb{V}_o \to \mathbb{R}$ is defined by
$$f_o(\boldsymbol{v}_o;\boldsymbol{\mu})=-\int_{\Gamma_{o,7}} \boldsymbol{v}_o\cdot\boldsymbol{n} \ ds.$$

In [ ]:
from dolfin import *
from rbnics import *

## 3. Affine decomposition

In order to obtain an affine decomposition, we recast the problem on a fixed, parameter _independent_, reference domain $\Omega$. As reference domain which choose the one characterized by $\mu_0 = 0.05$ which we generate through the generate_mesh notebook provided in the _data_ folder.
Then, we pull back the problem to the reference domain $\Omega$.

In [ ]:
@SCM()
@PullBackFormsToReferenceDomain()
@AffineShapeParametrization("data/bridge_vertices_mapping.vmp")

class Bridge(EllipticCoerciveProblem):
    
    #Default initialization of members
    @generate_function_space_for_stability_factor
    def __init__(self,V,**kwargs):
        # Call the standard initialization
        EllipticCoerciveProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=subdomains)
        self.ds = Measure("ds")(subdomain_data=boundaries)
        # ...
        
        self.nu = 0.3
        self.E = 1.0
        self.lambda_1 = self.E*self.nu / ((1.0 + self.nu)*(1.0 - 2.0*self.nu))
        self.lambda_2 = self.E / (2.0*(1.0 + self.nu))
        self._eigen_solver_parameters.update({
           "bounding_box_minimum": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert", "spectral_shift": 1.e-5, "linear_solver": "mumps"},
           "bounding_box_maximum": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert", "spectral_shift": 1.e5, "linear_solver": "mumps"},
           "stability_factor": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert", "spectral_shift": 1.e-5, "linear_solver": "mumps"}            
        })
        
    # Return custom problem name
    def name(self):
        return "Bridge"
    
    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_stability_factor
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = mu[1]
            return (theta_a0,)
        elif term == "f":
            theta_f0 = 1.0
            return (theta_f0,)
        elif term == "s":
            theta_s0 = 1.0
            return (theta_s0,)
        else:
            raise ValueError("Invalid term for compute_theta().")
            
    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_stability_factor
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2
        if term == "a":
            u = self.u
            elasticity = 2.0*lambda_2*inner(sym(nabla_grad(u)), sym(nabla_grad(v))) + lambda_1*tr(sym(nabla_grad(u)))*tr(sym(nabla_grad(v)))
            a0 = elasticity*dx
            return (a0,)
        elif term == "f":
            ds = self.ds
            u = self.u
            f0 = inner(Constant((0.0, -1.0)), v)*ds(7)
            return (f0,)
        elif term == "s":
            ds = self.ds
            u = self.u
            s0 = inner(Constant((0.0, -1.0)), v)*ds(7)
            return (s0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V.sub(1), Constant(0.0), self.boundaries, 1),
                  DirichletBC(self.V.sub(1), Constant(0.0), self.boundaries, 2),
                  DirichletBC(self.V, Constant((0.0, 0.0)), self.boundaries, 8),
                  DirichletBC(self.V, Constant((0.0, 0.0)), self.boundaries, 9)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(u, v)*dx + inner(grad(u), grad(v))*dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [ ]:
mesh = Mesh("data/bridge.xml")
subdomains = MeshFunction("size_t", mesh, "data/bridge_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/bridge_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P2)

In [ ]:
V = VectorFunctionSpace(mesh, "Lagrange", 2)

### 4.3. Allocate an object of the Bridge class

In [ ]:
problem = Bridge(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.05, 0.6),(0.1, 10.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a reduced basis method

In [ ]:
reduction_method = ReducedBasis(problem)
reduction_method.set_Nmax(20, SCM=10)
reduction_method.set_tolerance(1e-5, SCM=1e-3)

### 4.5. Perform the offline phase

In [ ]:
reduction_method.initialize_training_set(200, SCM=250)
reduced_problem = reduction_method.offline()

### 4.6. Perform an online solve

In [ ]:
online_mu = (0.09, 1.86)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem)

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(100, SCM=100)
reduction_method.error_analysis(filename="error_analysis")

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis(filename="speedup_analysis")

### 4.9. Perform an error and speedup analysis employing a smaller number of SCM constraints

In [ ]:
reduction_method.error_analysis(SCM=3, filename="error_analysis_SCM_3")
reduction_method.speedup_analysis(SCM=3, filename="speedup_analysis_SCM_3")

### 5. Assignments


1.  Change the model reduction technique to POD-Galerkin. Compare the results of the error analysis and speedup analysis for the two reduction techniques.